# Usage 

The Tomo NDt Library is a library that works on top of [Tomo Base](https://github.com/Tcraig088/tomobase). The purpose of this library is to extrapolate functions from 2D 3D to consider time dependent effects. In addition it allows storage of volume-time data in a compressed file allowin for convient storage of metadata and compression.

## Read Write 

Volume Time series data is read and written to disk and only accessed where required. To initialize an ndt data set provide a location to save data to and a volume to initialize its shape, pixelsize and metadata. Volume data can be written or read from a volume time series using the read and write_volume functions while specifying the time in seconds. 

In [ ]:
import tomobase.phantoms as phantoms
from  tomondt.data import VolumeNDt

vol =phantoms.get_nanocage()

vndt = VolumeNDt(path='new_file.vmf', volume=vol)
vndt.write_volume(vol, time=7.5)
vol = vndt.read_volume(time=7.5)

## Operators 
The libary abstracts tomography tasks into operations. These operations each transform one data type into another. A reconstruction is a tomographic procedure which transforms a 2D into a 3D sinogram. When working with time dependent data some operations that would typically transform data into 3D need to operate with 3D + the time dimension. 

An example is segmentation. A typical post processing of segmentation procedure will deform a 3D volume to create another 3D volume or labelled dataset. When conducted with time, this procedure must be performed on all volumes in a volume time series. 

The Tomo NDt library supports the following conversions on top of procedures already available by Tomo Base:

1. Deformation: Converts a volume into a volume time series by iteratively applying a deformation to a volume
2. Forward Projection: Converts a volume-time series to a temporally labelled Sinogram
3. Reconstruct: Converts a Sinogram into a volume time series
4. Segmentation/Post processing: Converts a volume time series into a volume time series.

In [ ]:
from tomobase import phantoms
from tomobase import processes
import numpy as np

from tomondt.operators import deform_ndt

times = np.linespace(1, 30, 30)
vol = phantoms.get_nanocage()
vndt = deform_ndt(vol, times, processes.beamdamage, knockon=0.1, elastic_deform=0.01, normalize=True)
# creates a time series with 30 time points

In [ ]:

from tomobase import tiltschemes
import numpy as np

from tomondt.operators import project_ndt

ts = tiltschemes.GRS(-70, 70, 0)
angles = np.array([ts.get_angle() for i in range(71)])
#times can be determined from the times of the volumes in vndt or by approximating the closest time to specified times 

sino = project_ndt(vndt, times, processes.beamdamage, knockon=0.1, elastic_deform=0.01, normalize=True)
# creates a time series with 30 time points

In [ ]:
import tomobase.processes as processes
import numpy as np

from tomondt.operators import reconstruct_static_ndt

# A dynamic api will be included seperately
vndt = reconstruct_static_ndt(sino, 50, 1, processes.astra_reconstruct, method='sirt', iterations=100)


In [ ]:
import tomobase.processes as processes
import numpy as np

from tomondt.operators import segment_ndt


vndt2 = segment_ndt(vndt, processes.poisson_noise, 0.1)